In [1]:
import os

In [2]:
%pwd

'c:\\Users\\arbas\\Documents\\Projects\\Machine-Learning\\EndToEnd_Deployed\\Project 4\\research'

In [3]:
os.chdir('../')

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class Data_Ingestion_Config:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [16]:
from src.Project4.constants import *
from src.Project4.utils.common import read_yaml,create_directories

In [17]:
class Configuration_Manager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH,params_file_path = PARAMS_FILE_PATH,schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->Data_Ingestion_Config:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = Data_Ingestion_Config(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [18]:
import os
import urllib.request as request
import zipfile
from src.Project4.utils.common import get_size, logger

In [19]:
class Data_Ingestion:
    def __init__(self,config: Data_Ingestion_Config):
        self.config = config
        
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header = request.urlretrieve(url=self.config.source_URL,filename=self.config.local_data_file)
            logger.info(f'Successfully downloaded data {filename} with following info: {header} at: {self.config.local_data_file}')
        else:
            logger.info(f'Data already exists of size {get_size(Path(self.config.local_data_file))}')
    
    def extract_zip_data(self):
        ''' Extracts the zip file
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = Configuration_Manager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = Data_Ingestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_data()
except Exception as e:
    raise e

2024-02-24 11:13:35,538 - src_logger - INFO - Yaml file:config\config.yaml loaded successfully
2024-02-24 11:13:35,542 - src_logger - INFO - Yaml file:params.yaml loaded successfully
2024-02-24 11:13:35,544 - src_logger - INFO - Yaml file:schema.yaml loaded successfully
2024-02-24 11:13:35,547 - src_logger - INFO - Created directory at: artifacts
2024-02-24 11:13:35,549 - src_logger - INFO - Created directory at: artifacts/data_ingestion
2024-02-24 11:13:37,194 - src_logger - INFO - Successfully downloaded data artifacts/data_ingestion/data.zip with following info: Connection: close
Content-Length: 3430
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4c665d358601dc38a7863a4318d27efca4fc56a53f28d43470b8ef8b60777e1a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F51E:25E141:4869C:6DDB9:65D9